## Import Modules

In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import glob
# import matplotlib.pyplot as plt
import pymannkendall as mk
import geopandas as gpd
import salem
from esmtools.grid import convert_lon 
from utils import *

# Filter Warnings
import warnings
warnings.filterwarnings('ignore')

# options
xr.set_options(keep_attrs=True)

## Directory and shapefiles

In [2]:
# Mudar o diretorio
os.chdir('/home/luiz/climate_change/ETCCDI_Extremes/Anual/')

In [3]:
# Shape
shp = '/home/luiz/Documentos/Shapefiles/South_America_ecorreg_diss/South_America_ecorregions_dissolve.shp'
df_shp = gpd.read_file(shp)

In [4]:
#!ls *.nc

In [5]:
# Definindo o nome do Bioma Faltante
# df_shp.at[5, 'BIOME_NAME'] = 'Rock and Ice'
# df_shp

In [6]:
# Pegar lista com valores unicos dos biomas
biomes = list(df_shp.drop([5]).BIOME_NAME.unique())
ETCCDI_index = glob.glob('*nc')
ETCCDI_index.sort()

# Calculo das Tendencias dos Indices Extremos por Bioma

In [7]:
%%time

start_date = '1979-01-01'
end_date = '2020-01-01'
freq = '1YS'
trend_list = []
sig_list = []
biomes_list = []
index_list = []
sign_list = []


for biome in biomes:
    for file_nc in ETCCDI_index:
        # Abrir os bancos de dados
        ds = xr.open_dataset(file_nc, decode_timedelta=False)
        ds = fix_encoding_time(ds, start=start_date, end=end_date, freq=freq)
        ds = convert_lon(ds, coord=get_lon_name(ds))
        # Recortar por cada bioma
        ds = mask_dataset(ds, shapefile_path=shp, col_shapefile='BIOME_NAME', query=biome)
        
        # Remover variavel 'time_bnds'
        if 'time_bnds' in ds:
            ds = ds.drop_vars('time_bnds')
        else:
            pass
        # Pegar os valores
        df = get_stats(ds, getvar(ds))
        df.set_index('Time', inplace=True)
        
        # Calcular as tendencias e inclinação pelo teste de Mann-Kendall e Theil-Sen
        res, trend_line = mann_kendall_test(df[df.columns[1]]) # Weighted Mean
        trend, sig, sign = res.slope, res.p, res.trend
        
        # Criar lista slope
        trend_list.append(trend)
        # Criar lista sig
        sig_list.append(sig)
        # Criar lista
        sign_list.append(sign)
        # Criar lista biomes
        biomes_list.append(biome)
        # Criar lista biomes
        index_list.append(file_nc.split('_')[0])
        
        print(f'Index: ', file_nc.upper().split('_')[0], ' Biome: ', biome)
        
# Criar dataframe
db = pd.DataFrame({'Index_ETCCDI':index_list, 
                  'Trend':trend_list, 
                  'Sig':sig_list, 
                  'Bioma':biomes_list,
                  'Sign':sign_list})

Index:  CDD  Biome:  Deserts & Xeric Shrublands
Index:  CSDI  Biome:  Deserts & Xeric Shrublands
Index:  CWD  Biome:  Deserts & Xeric Shrublands
Index:  DTR  Biome:  Deserts & Xeric Shrublands
Index:  FD  Biome:  Deserts & Xeric Shrublands
Index:  PRCPTOT  Biome:  Deserts & Xeric Shrublands
Index:  R10MM  Biome:  Deserts & Xeric Shrublands
Index:  R20MM  Biome:  Deserts & Xeric Shrublands
Index:  R30MM  Biome:  Deserts & Xeric Shrublands
Index:  R95P  Biome:  Deserts & Xeric Shrublands
Index:  R99P  Biome:  Deserts & Xeric Shrublands
Index:  RX1DAY  Biome:  Deserts & Xeric Shrublands
Index:  RX5DAY  Biome:  Deserts & Xeric Shrublands
Index:  SDII  Biome:  Deserts & Xeric Shrublands
Index:  TN10P  Biome:  Deserts & Xeric Shrublands
Index:  TN90P  Biome:  Deserts & Xeric Shrublands
Index:  TNN  Biome:  Deserts & Xeric Shrublands
Index:  TNX  Biome:  Deserts & Xeric Shrublands
Index:  TX10P  Biome:  Deserts & Xeric Shrublands
Index:  TX90P  Biome:  Deserts & Xeric Shrublands
Index:  TXN  

In [8]:
db.to_excel('trends-doc-Biomes.xlsx', index=False)
#df.query('Index == "rx1day"')

# Cálculo das Tendencias dos Índices Extremos pelas regioes do IPCC

## IPCC Regions Shapefile

In [9]:
shp_IPCC = '/home/luiz/Documentos/Shapefiles/IPCC_Regions_SA/IPCC_Regions_SA.shp'
df_shp_IPCC = gpd.read_file(shp_IPCC)

In [10]:
# Pegar lista com valores unicos
ipcc_regions = list(df_shp_IPCC.Acronym.unique())

## Cálculo

In [11]:
%%time

start_date = '1979-01-01'
end_date = '2020-01-01'
freq = '1YS'
trend_list = []
sig_list = []
ipcc_list = []
index_list = []
sign_list = []


for region in ipcc_regions:
    for file_nc in ETCCDI_index:
        # Abrir os bancos de dados
        ds = xr.open_dataset(file_nc, decode_timedelta=False)
        ds = fix_encoding_time(ds, start=start_date, end=end_date, freq=freq)
        ds = convert_lon(ds, coord=get_lon_name(ds))
        # Recortar por cada região do IPCC
        ds = mask_dataset(ds, shapefile_path=shp_IPCC, col_shapefile='Acronym', query=region)
        
        # Remover variavel 'time_bnds'
        if 'time_bnds' in ds:
            ds = ds.drop_vars('time_bnds')
        else:
            pass
        # Pegar os valores
        df = get_stats(ds, getvar(ds))
        df.set_index('Time', inplace=True)
        
        # Calcular as tendencias e inclinação pelo teste de Mann-Kendall e Theil-Sen
        res, trend_line = mann_kendall_test(df[df.columns[1]]) # Weighted Mean
        trend, sig, sign = res.slope, res.p, res.trend
        
        # Criar lista slope
        trend_list.append(trend)
        # Criar lista sig
        sig_list.append(sig)
        # Criar lista
        sign_list.append(sign)
        # Criar lista ipcc regions
        ipcc_list.append(region)
        # Criar lista biomes
        index_list.append(file_nc.split('_')[0])
        
        print(f'Index: ', file_nc.upper().split('_')[0], ' IPCC Region: ', region)
        
# Criar dataframe
db = pd.DataFrame({'Index_ETCCDI':index_list, 
                  'Trend':trend_list, 
                  'Sig':sig_list, 
                  'IPCC_Region':ipcc_list,
                  'Sign':sign_list})

Index:  CDD  IPCC Region:  NWS
Index:  CSDI  IPCC Region:  NWS
Index:  CWD  IPCC Region:  NWS
Index:  DTR  IPCC Region:  NWS
Index:  FD  IPCC Region:  NWS
Index:  PRCPTOT  IPCC Region:  NWS
Index:  R10MM  IPCC Region:  NWS
Index:  R20MM  IPCC Region:  NWS
Index:  R30MM  IPCC Region:  NWS
Index:  R95P  IPCC Region:  NWS
Index:  R99P  IPCC Region:  NWS
Index:  RX1DAY  IPCC Region:  NWS
Index:  RX5DAY  IPCC Region:  NWS
Index:  SDII  IPCC Region:  NWS
Index:  TN10P  IPCC Region:  NWS
Index:  TN90P  IPCC Region:  NWS
Index:  TNN  IPCC Region:  NWS
Index:  TNX  IPCC Region:  NWS
Index:  TX10P  IPCC Region:  NWS
Index:  TX90P  IPCC Region:  NWS
Index:  TXN  IPCC Region:  NWS
Index:  TXX  IPCC Region:  NWS
Index:  WSDI  IPCC Region:  NWS
Index:  CDD  IPCC Region:  NSA
Index:  CSDI  IPCC Region:  NSA
Index:  CWD  IPCC Region:  NSA
Index:  DTR  IPCC Region:  NSA
Index:  FD  IPCC Region:  NSA
Index:  PRCPTOT  IPCC Region:  NSA
Index:  R10MM  IPCC Region:  NSA
Index:  R20MM  IPCC Region:  NSA
Ind

In [12]:
# Export excel
db.to_excel('/home/luiz/Jupyter_Notebook/Python_Notebook/Climate_Extremes_DOC/trends-doc-IPCCRegions.xlsx',
           index=False)